In [7]:
import torch, gc, random, datasets
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
%load_ext memory_profiler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [8]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf
 
      tf.random.set_seed(seed)
 
set_seed(42)

In [9]:
df = pd.read_csv("./like_data.csv")

In [15]:
def LinearRegression_with_Transformer(model_name: str, Data: pd.Series, Target:pd.Series, test_size: np.float64, max_length: int, num_labels: int, num_epochs: int):

  # Make data
  X = Data
  y = Target


  # Split Data
  X_train, X_test, y_train, y_test = train_test_split(X.tolist(), y, test_size=test_size)

  # Call the Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # Encode the text
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)



  class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          item["labels"] = float(item["labels"])
          return item

      def __len__(self):
          return len(self.labels)

  # convert our tokenized data into a torch Dataset
  train_dataset = MakeTorchData(train_encodings, y_train.ravel())
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())


  # Call Model
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels).to("cuda") # np.log(1000)

  # def inverse_sigmoid(logits):
  #   return -np.log((1.0 / (logits + 1e-8)) - 1.0)

  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import r2_score

  def compute_metrics_for_regression(eval_pred):
      logits, labels = eval_pred
      # print("Logits:", logits[0:5])
      # print("Labels:", labels[0:5])
      # logits = inverse_sigmoid(logits)
      labels = labels.reshape(-1, 1)
      
      print("Logits:", logits[0:5])
      print("Labels:", labels[0:5])
      # print("Labels:", labels)

      mse = mean_squared_error(labels, logits)
      rmse = mean_squared_error(labels, logits, squared=False)
      mae = mean_absolute_error(labels, logits)
      r2 = r2_score(labels, logits)
      # smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
      single_squared_errors = ((logits - labels).flatten()**2).tolist()
      accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
      
      return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "accuracy": accuracy}

  # Specifiy the arguments for the trainer  
  training_args = TrainingArguments(
      output_dir='./results',          # output directory
      num_train_epochs=num_epochs,     # total number of training epochs
      per_device_train_batch_size=64,   # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      weight_decay=0.01,               # strength of weight decay
      learning_rate=2e-5,
      # logging_dir='./logs',            # directory for storing logs
      save_total_limit=10,
      load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
      metric_for_best_model = 'accuracy',    # select the base metrics
      evaluation_strategy="epoch",
      save_strategy="epoch",
  ) 

  # Call the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=valid_dataset,          # evaluation dataset
      compute_metrics=compute_metrics_for_regression,     # the callback that computes metrics of interest
  )

  # Train the model
  trainer.train()
  
  # Call the summary
  trainer.evaluate()



  return trainer, model

In [16]:
%%time
%%memit
bert_trainer, bert_model = LinearRegression_with_Transformer(model_name = 'google-bert/bert-base-multilingual-cased', 
                                                                 Data = df.posts.astype(str), 
                                                                 Target = df.likes.astype(float), 
                                                                 test_size = 0.2, 
                                                                 max_length = 512, 
                                                                 num_labels = 1, 
                                                                 num_epochs = 100)

C:\Users\samuel\AppData\Local\Temp\ipykernel_29864\2243282803.py:35: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  train_dataset = MakeTorchData(train_encodings, y_train.ravel())
C:\Users\samuel\AppData\Local\Temp\ipykernel_29864\2243282803.py:36: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: Torch not compiled with CUDA enabled